# Beamline 8.3.2 Reconstruction

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import sys
import os
import multiprocessing as mp
os.environ['NUMEXPR_MAX_THREADS'] = str(mp.cpu_count()) # to avoid numexpr warning
os.environ['TOMOPY_PYTHON_THREADS'] = str(mp.cpu_count()) # to avoid numexpr warning
import time
import numpy as np
import tomopy
import dxchange
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, fixed, IntSlider
from ipyfilechooser import FileChooser
sys.path.append('backend')
import ALS_recon_functions as als
import ALS_recon_helper as helper
use_gpu = als.check_for_gpu()

## Choose Data

In [ ]:
dataDir = "/alsuser" if os.path.exists("/alsuser") else os.getcwd() 
#dataDir = r"\\data832.lbl.gov\bl832data-raw" # if on beamline machines
file_chooser = FileChooser(dataDir)
file_chooser.filter_pattern = '*.h5' # only show .h5 files
file_chooser.title = f'Choose data file'
display(file_chooser)

## Find Center of Rotation (COR)
##### Check COR by manually aligning 0 and 180 degree projections

In [ ]:
if file_chooser.selected is None:
    print("No datafile selected. Click Select to choose a datafile")
    sys.exit()
metadata = als.read_metadata(file_chooser.selected, print_flag = True)
init_cor, cor_tomo = als.auto_find_cor(file_chooser.selected) # cross-correlation based COR estimate
first_proj, last_proj_flipped = cor_tomo[0], np.fliplr(cor_tomo[1])

axs, img, cor_sliders, cor_out = als.plot_0_and_180_proj_diff(first_proj,
                                                              last_proj_flipped,
                                                              init_cor=init_cor,
                                                              # yshift=True, # shows extra slider to shift proj up/down
                                                              # continuous_update=False # can set continuous update=False to reduce lag (only updates on slider release) 
                                                              fignum=0)
                                                            
display(cor_sliders,cor_out)

##### Unfold Cell Below to Plot Normalized Projections and Sinograms
##### Optional, for visualization only - this cell loads all projections, so will take a minute

In [ ]:
%%time

###### SET ME! ######
downsample_factor = 4
convert360to180 = False
###### SET ME! ######

tomo, angles = als.read_data(file_chooser.selected, downsample_factor=downsample_factor)

if metadata['angularrange'] > 300 and convert360to180: # convert 360 to 180
    print("Detected 360 degree acquisition - will convert sinograms to 180 degrees")
    COR = cor_sliders.children[0].value

    if tomo.shape[0]%2>0:
        tomo = als.sino_360_to_180(tomo[0:-1,:,:], overlap=int(np.round((tomo.shape[2]//2-COR/downsample_factor-.5))*2), rotation='right')           
    else:
        tomo = als.sino_360_to_180(tomo[:,:,:], overlap=int(np.round((tomo.shape[2]//2-COR/downsample_factor))*2), rotation='right')                       
    angles = angles[:tomo.shape[0]]


if plt.fignum_exists(2): plt.close(2)
tomo_fig, tomo_axs = plt.subplots(1,2,num=2,figsize=(6,3))
proj_img = tomo_axs[0].imshow(tomo[0],cmap='gray',vmin=0,vmax=1)
tomo_axs[0].set_title(f"Projection {0}: angle = {0} deg")
sino_img = tomo_axs[1].imshow(tomo[:,tomo.shape[1]//2,:],cmap='gray',vmin=0,vmax=1)
tomo_axs[1].set_title(f"Sinogram slice {tomo.shape[1]//2}")
plt.tight_layout()


def set_proj(proj_num):
    proj_img.set_data(tomo[proj_num,:,:])
    tomo_axs[0].set_title(f"Projection {proj_num}: angle = {angles[proj_num]} deg")
def set_sino(sino_num):
    sino_img.set_data(tomo[:,sino_num,:])
    tomo_axs[1].set_title(f"Sinogram slice {sino_num}")
    

proj_slider = widgets.interactive(set_proj, path=widgets.fixed(file_chooser.selected),
                                  proj_num=widgets.IntSlider(description='Projection', style={'description_width': 'initial'}, layout=widgets.Layout(width='50%'),
                                                             min=0, max=tomo.shape[0]-1,
                                                             step=1, value=0,
                                                             continuous_update=True))
sino_slider = widgets.interactive(set_sino, path=widgets.fixed(file_chooser.selected),
                                  sino_num=widgets.IntSlider(description='Sinogram Slice', style={'description_width': 'initial'}, layout=widgets.Layout(width='50%'),
                                                             min=0, max=tomo.shape[1]-1,
                                                             step=1, value=tomo.shape[1]//2,
                                                             continuous_update=True))

clim_slider = widgets.interactive(als.set_clim, img=widgets.fixed([proj_img,sino_img]),
                                       clims=widgets.FloatRangeSlider(description='Projection Color Scale', style={'description_width': 'initial'}, layout=widgets.Layout(width='50%'),
                                                                      min=np.minimum(tomo.min(),-0.1), max=np.maximum(tomo.max(),1.1),
                                                                      step=1/500, value=[0.1, 1],
                                                                      readout=True, readout_format='.2e'))

display(proj_slider)
display(sino_slider)
display(clim_slider) # show color scale widget

##### Unfold cell below for description of reconstruction parameters

#### 1. Alignment
* **COR:**
Initializes to value found above, but can fine tune if desred. With mismatched COR, arc/shadow artifacts appear in image. Adjust COR to minimize artifacts.

#### 2. Resolution
*Note: On NERSC, reconstruction takes ~2-4 sec/slice with GPU, ~30 sec/slice on CPU at full resolution (1313 ang x 2560 rays).*

* **Angle Downsampling:**
Downsampling angles by 2x reduces reconstruction time by ~2x, but may introduce streaks through center.

* **Projection Downsampling:**
Downsampling projections by 2x reduces spatial resolution by 2x, reconstruction time by ~4x (minus loading time).

#### 3. Ring Removal
*Note: strong stripe removal takes time, can reduce resolution and sometimes even introduce artifacts too, so only use as much as you need to remove visible rings

* **Small Ring Size:**
Window size of the median filter to remove small stripes. Larger is stronger but takes longer. Set to appx width of small stripes. Always choose odd value, set to 1 to turn off.

* **Large Ring Size:**
Window size of the median filter to remove large stripes. Set to appx width of large stripes -- should be larger value than Small Ring Size. Always choose odd value, set to 1 to turn off.

* **Ring Removal SNR:**
Sensitivity of large stripe detection method. Smaller is more sensitive. No affect on small stripes. Recommended values: 1.1 -- 3.0.

## Run Following Cell to Generate an Interface to Adjust Reconstruction Parameters
##### FYI: if you want to hide the code, click the blue bar next to the cell after running it

In [ ]:
# Initial recon needed to get size/colorscale of plots
cor = cor_sliders.children[0].value
recon_init, tomo = helper.reconstruct(file_chooser.selected, angles_ind=None, slices_ind=slice(metadata["numslices"]//2,metadata["numslices"]//2+1,1), COR=cor, use_gpu=use_gpu)
clim_init = [np.percentile(recon_init,1),np.percentile(recon_init,99)]

# Reconstructions/sinograms figures
if plt.fignum_exists(1): plt.close(1)
recon_comparison_fig, recon_comparison_axs = plt.subplots(1,2,num=1,figsize=(6,3),sharex=True,sharey=True)
recon_comparison_img = [None, None]
recon_comparison_img[0] = recon_comparison_axs[0].imshow(recon_init.squeeze(),cmap='gray')
recon_comparison_axs[0].set_ylabel('Slice Reconstruction',color='k', fontsize=10)
recon_comparison_axs[0].set_title('Recon Parameters',fontweight="bold",fontsize=12)
recon_comparison_axs[0].patch.set_edgecolor('black')  
recon_comparison_axs[0].patch.set_linewidth(4)  
recon_comparison_img[1] = recon_comparison_axs[1].imshow(recon_init.squeeze(),cmap='gray')
recon_comparison_axs[1].set_title("Only for comparison\n Parameters will not be used",color='r', fontsize=10)
plt.tight_layout()


recon_parameter_widgets, recon_parameters_tab, recon_output = helper.reconstruction_parameter_options(file_chooser.selected,
                                                                cor,use_gpu,recon_comparison_img[0],None,None)
comparison_parameter_widgets, comparison_parameters_tab, comparison_output = helper.reconstruction_parameter_options(file_chooser.selected,
                                                                cor,use_gpu,recon_comparison_img[1],None,None)


recon_clim_slider = widgets.interactive(als.set_clim, img=widgets.fixed(recon_comparison_img),
                              clims=widgets.FloatRangeSlider(description='Recon Color Scale', style={'description_width': 'initial'}, layout=widgets.Layout(width='50%'),
                                                                   min=recon_init.min(), max=recon_init.max(),
                                                                   step=(recon_init.max()-recon_init.min())/500, value=clim_init,
                                                                   readout=True, readout_format='.2e'))

# Haven't played around with layout too much. This part could probably be much fancier
recon_header_widget = widgets.HTML(value=f"<b><font size=3>Recon Parameters</b>",layout=widgets.Layout(justify_content="center"))
comparison_header_widget = widgets.HTML(value=f"<b><font size=3>Comparison</b>",layout=widgets.Layout(justify_content="center"))

box_layout = widgets.Layout(width='100%',
                            height='100%',
                            margin='0,1%,0,1%')
box = widgets.Box(children=[widgets.VBox([recon_header_widget,recon_parameters_tab]),
                            widgets.VBox([comparison_header_widget,comparison_parameters_tab])]
                            ,layout=box_layout)

# Display
display(box, recon_output, comparison_output) # show recon parameter widgets
display(recon_clim_slider) # show recon color scale widget

## Choose Number of Slices to Reconstruct and Location to Save Images
##### Unfold cell below for description of reconstruction methods

**Gridrec:** fastest, recommended on Cori CPU nodes

**FBP:** better image quality, recommended on Cori GPU or Perlmutter CPU/GPU nodes

**CGLS:** A little better image quality than FBP. For 20 iters (default), roughly same speed as FBP. Recommended on Cori GPU or Perlmutter GPU nodes

In [ ]:
save_file_chooser = FileChooser(als.get_scratch_path())
save_file_chooser.show_only_dirs = True
save_file_chooser.title = f'Choose where to save reconstruction'

slices_header_widget = widgets.Label(value="Choose which slices to reconstruct",layout=widgets.Layout(justify_content="center"))

start_slice_widget = widgets.BoundedIntText(description='Start Slice',
                                min=0,
                                max=metadata['numslices']-1,
                                step=1,
                                value=0,
    style={'description_width': 'initial'} # this makes sure description text doesn't get cut off
)
stop_slice_widget = widgets.BoundedIntText(description='Stop Slice',
                                min=0,
                                max=metadata['numslices']-1,
                                step=1,
                                value=metadata['numslices']-1,
    style={'description_width': 'initial'} # this makes sure description text doesn't get cut off
)
recon_method_widget = widgets.Dropdown(
    options=[("Default",'default'),
             ("Gridrec",'gridrec'),
             ("FBP",'fbp'),
             ("CGLS",'cgls')],
    value='default',
    description='Reconstruction Method:',
    style={'description_width': 'initial'} # this makes sure description text doesn't get cut off
)
slices_box = widgets.VBox([slices_header_widget,start_slice_widget,stop_slice_widget])
display(save_file_chooser)
display(slices_box)
display(recon_method_widget)

## Populate Parameters of Reconstruction with "Recon Parameters"

In [ ]:
if save_file_chooser.selected_path is None:
    raise TypeError("You forgot to set a save directory")
data_settings = {
    "output_path": save_file_chooser.selected_path, # existing folder where you can write lots of data
    "data_path": file_chooser.selected, # don't change
    "name": os.path.splitext(file_chooser.selected_filename)[0], # don't change
    "start_slice": start_slice_widget.value,
    "stop_slice": stop_slice_widget.value,
    "angles_ind": slice(0,None,recon_parameter_widgets['angle_downsample'].value), # use every angle 
    "proj_downsample": recon_parameter_widgets['proj_downsample'].value,
}

preprocess_settings = {
    "snr": recon_parameter_widgets['ring']['sarepy_snr'].value, 
    "la_size": recon_parameter_widgets['ring']['sarepy_large'].value,
    "sm_size": recon_parameter_widgets['ring']['sarepy_small'].value,
    "outlier_diff_1D": recon_parameter_widgets['additional']['outlier_diff'].value,
    "outlier_sizef_1D": recon_parameter_widgets['additional']['outlier_size'].value,
    "minimum_transmission": recon_parameter_widgets['additional']['min_transmission'].value
}

postprocess_settings = {
    "ringSigma": recon_parameter_widgets['ring']['ringSigma'].value,
    "ringLevel": recon_parameter_widgets['ring']['ringLevel'].value
}

recon_settings = {
    "method": recon_method_widget.value,
    "COR": recon_parameter_widgets['cor'].value,
    "fc": recon_parameter_widgets['fc'].value, 
    "use_gpu": use_gpu
}

settings = {"data": data_settings, "preprocess": preprocess_settings, "postprocess": postprocess_settings, "recon": recon_settings}
for subset in settings:
   print('\n' + subset.upper())
   for key in settings[subset]:
       print(f"{key}: {settings[subset][key]}") 

## Reconstruct and Save 3D Volume in Notebook
##### Approximate reconstruction time per 50 slices (using default method)
###### - Perlmutter Exclusive GPU node: ~5-10 sec
###### - Perlmutter Exclusive CPU node: ~10-20 sec
###### - Cori Shared CPU node: ~30-200 sec
##### *Note: For more efficient workflow, submit reconstruction to NERSC batch job (see below)*

In [ ]:
nchunk = 50 # Balance between available cpus and memory (larger value can be more parallelized but uses more memory). 50 was empirically chosen on Perlmutter exclusive node, though 100 was more or less the same.
save_dir = os.path.join(settings["data"]["output_path"],settings["data"]["name"])
if not os.path.exists(save_dir): os.makedirs(save_dir)
save_name = os.path.join(save_dir,settings["data"]["name"])
for i in range(np.ceil((settings["data"]['stop_slice']-settings["data"]['start_slice'])/nchunk).astype(int)):
    start_iter = settings["data"]['start_slice']+i*nchunk
    stop_iter = np.minimum(start_iter+nchunk,settings["data"]['stop_slice'])
    print(f"Starting recon of slices {start_iter}-{stop_iter}...",end=' ')
    tic = time.time()

    recon,_ = helper.reconstruct(path=settings["data"]["data_path"],
                           angles_ind=settings["data"]['angles_ind'],
                           slices_ind=slice(start_iter,stop_iter,1),
                           COR=settings["recon"]["COR"],
                           method=settings["recon"]["method"],
                           proj_downsample=settings["data"]["proj_downsample"],
                           fc=settings["recon"]["fc"],
                           preprocessing_settings=settings["preprocess"],
                           postprocessing_settings=settings["postprocess"],
                           use_gpu=settings["recon"]["use_gpu"])

    print(f"Finished: took {time.time()-tic} sec. Saving files...")
    dxchange.write_tiff_stack(recon, fname=save_name, start=start_iter)
print("Done")

## Reconstruct and Save 3D Volume with NERSC Batch Job
##### 1. The cell below prepares the config script needed to submit a job

In [ ]:
import ALS_batch_recon as batch_recon

configs_dir, config_script_name = batch_recon.create_batch_script(settings)   
# list configs_dir
print(f"Contents of batch jobs configs directory: {configs_dir}") 
os.system(f"ls {configs_dir}")
print('')

##### 2. Run the following cell to submit the batch job

In [ ]:
# submit batch job config you just created
os.system(f"sbatch {os.path.join(configs_dir,config_script_name)}")
# see all current batch jobs (including this interavtive jupyter session)
print("List of current batch jobs:")
os.system("sqs")

##### To cancel a batch job, run this cell with the right Job ID

In [ ]:
# eq. scancel 2800193
os.system("scancel 0000000")

## Create NERSC Batch Jobs for Multiple Datasets
##### Choose directory, will use the same settings for all datasets in that directory

In [ ]:
dataDir = "/alsuser" if os.path.exists("/alsuser") else os.getcwd() 
batch_file_chooser = FileChooser(dataDir)
batch_file_chooser.show_only_dirs = True
batch_file_chooser.title = f'Choose data directory'
slice_range_widget = widgets.RadioButtons(
    options=[('reconstruct all slices',0),('use EXACT same range for all',1)],
    value=0, # default
    description='Slice Range:',
    disabled=False
)
cor_widget = widgets.RadioButtons(
    options=[('use auto-COR finder for all',0), ('use EXACT same COR for all',1)],
    value=0, # default
    description='COR:',
    disabled=False
)
display(batch_file_chooser)
display(slice_range_widget)
display(cor_widget)

##### Create batch jobs for all .h5 files

In [ ]:
import ALS_batch_recon as batch_recon

for file in os.listdir(batch_file_chooser.selected_path):
    if file.endswith(".h5"):
        # update data path and name
        settings["data"]["data_path"] = os.path.join(batch_file_chooser.selected_path,file)
        settings["data"]["name"] = os.path.splitext(file)[0]
        metadata = als.read_metadata(os.path.join(batch_file_chooser.selected_path,file), print_flag = False)
        # change slice range, if necessary
        if slice_range_widget.value == 0:
            settings["data"]["start_slice"] =  0
            settings["data"]["stop_slice"] =  metadata['numslices']-1
        # change COR to auto, if necessary
        if cor_widget.value == 1:
            settings["recon"]["COR"] =  None
        
        # create template
        configs_dir,_ = batch_recon.create_batch_script(settings)   

##### Submit all batch jobs

In [ ]:
def submit_callback(b):
    with output:
        for config_script_name in os.listdir(configs_dir):
            if config_script_name.startswith("config_") and config_script_name.endswith(".sh"):
                os.system(f"sbatch {os.path.join(configs_dir,config_script_name)}")
        submit_button.layout.visibility = 'hidden'
        cancel_button.layout.visibility = 'hidden'
        print("List of current batch jobs:")
        # os.system("sqs")
def cancel_callback(b):
    with output:
        submit_button.layout.visibility = 'hidden'
        cancel_button.layout.visibility = 'hidden'
        print("List of current batch jobs:")
        # os.system("sqs")
label = widgets.HTML(value="<font color='red'><font size=4>WARNING: THIS WILL LAUNCH MANY MANY NERSC BATCH JOBS -- USE RESPONSIBLY!!!")
submit_button = widgets.Button(
    description='Submit',
    disabled=False,
    button_style='success',
)
cancel_button = widgets.Button(
    description='Cancel',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    on_click=cancel_callback
)
output = widgets.Output()
button_box = widgets.VBox([label, widgets.HBox([submit_button, cancel_button])])
display(button_box,output)

submit_button.on_click(submit_callback)
cancel_button.on_click(cancel_callback)

## Optional: SVMBIR reconstruction (for better image quality)

## Choose SVMBIR parameters
##### Unfold cell below for description of SVMBIR parameters

**p**: smoothing parameter for large grayscale differences (ie contrasting pixels).
<br> 1 = Total Variation: preserves edges but can look unrealistic ("cartoonish")
<br> 2 = Tikhonov regularization: looks natural but reduces resolution
<br> Typically a compromise between 1 and 2 works best

**q**: smoothing parameter for small grayscale differences (ie. similar pixels). Between p and 2

**T**: transition value from q to p (in pixel grayscale difference). Must be > 0

**sharpness**: strength of smoothing -- lower is more smooth. +1 change is equivalent to +6 dB snr. Can typically just change one or the other

**snr_dB**: strength of smoothing -- lower is more smooth. +6 dB is equivalent to +1 sharpness. Can typically just change one or the other

In [ ]:
import svmbir
svmbir_header_widget = widgets.Label(value="Choose SVMBIR parameters",layout=widgets.Layout(justify_content="center"))
blank_widget = widgets.Label(value="",layout=widgets.Layout(justify_content="center"))
p_widget = widgets.BoundedFloatText(description='p',
                                min=1,
                                max=2,
                                step=0.01,
                                value=1.2,
    style={'description_width': 'initial'} # this makes sure description text doesn't get cut off
)
q_widget = widgets.BoundedFloatText(description='q',
                                min=1,
                                max=2,
                                step=0.01,
                                value=2,
    style={'description_width': 'initial'} # this makes sure description text doesn't get cut off
)
T_widget = widgets.BoundedFloatText(description='T',
                                min=0.01,
                                max=10,
                                step=0.01,
                                value=0.1,
    style={'description_width': 'initial'} # this makes sure description text doesn't get cut off
)
sharpness_widget = widgets.BoundedFloatText(description='sharpness',
                                min=-10,
                                max=10,
                                step=0.1,
                                value=0,
    style={'description_width': 'initial'} # this makes sure description text doesn't get cut off
)
snr_dB_widget = widgets.BoundedFloatText(description='snr_dB',
                                min=0,
                                max=80,
                                step=1,
                                value=40,
    style={'description_width': 'initial'} # this makes sure description text doesn't get cut off
)
svmbir_slice_widget = widgets.BoundedIntText(description='Slice number',
                                min=0,
                                max=metadata['numslices']-1,
                                step=1,
                                value=metadata['numslices']//2,
    style={'description_width': 'initial'} # this makes sure description text doesn't get cut off
)

svmbir_box = widgets.HBox([
                        widgets.VBox([svmbir_slice_widget, svmbir_header_widget,p_widget,q_widget]),
                        widgets.VBox([blank_widget, blank_widget,T_widget,sharpness_widget,snr_dB_widget])
])
display(svmbir_box)

## Run SVMBIR on a Single Slice
##### Note: SVMBIR is MUCH slower. At full resolution (1313 ang x 2560 rays), takes ~2-4 min/slice with all 128 exclusive node threads (uses CPU only).
##### Downsampling by 2x reduces time by ~4x

In [ ]:
%%time
slices_ind = slice(svmbir_slice_widget.value,svmbir_slice_widget.value+1,1)

tic = time.time()

tomo, angles = als.read_data(settings["data"]["data_path"],
                             proj=settings["data"]["angles_ind"],
                             sino=slices_ind,
                             downsample_factor=settings["data"]["proj_downsample"],
                             preprocess_settings=settings["preprocess"],
                             postprocess_settings=settings["postprocess"])

astra_recon = als.astra_fbp_recon(tomo, angles,
                              COR=settings["recon"]["COR"]/settings["data"]["proj_downsample"],
                              fc=settings["recon"]["fc"],
                              gpu=settings["recon"]["use_gpu"])
astra_recon = als.mask_recon(astra_recon)
print(f"Finished Astra recon, took {time.time()-tic} sec. Starting SVBMIR...")

settings["svmbir_settings"] = {
    "p": p_widget.value, 
    "q": q_widget.value, 
    "T": T_widget.value,
    "sharpness": sharpness_widget.value, 
    "snr_dB": snr_dB_widget.value,
    "max_iter": 100,
    "COR": settings['recon']['COR']/settings["data"]["proj_downsample"]
}

tic = time.time()

svmbir_recon = als.svmbir_recon(tomo,angles,**settings["svmbir_settings"])
svmbir_recon = als.mask_recon(svmbir_recon)

print(f"Finished SVMBIR recon, took {time.time()-tic} sec")
print('')
print('This took:')

##### Compare Astra FBP and SVMBIR Reconstructions

In [ ]:
img, axs, clim_slider = als.plot_recon_comparison(astra_recon, svmbir_recon, titles=['FBP','SVMBIR'], fignum=5, figsize=4)
display(clim_slider)

##### Create SVMBIR batch job script

In [ ]:
import ALS_batch_recon as batch_recon

with open (batch_recon.get_batch_template(algorithm="svmbir"), "r") as t:
    template = t.read()

s = os.popen("echo $NERSC_HOST")
out = s.read()
N = 40 if 'cori' in out else 10
    
configs_dir = Path(os.path.join(settings["data"]["output_path"],"configs/"))
if not configs_dir.exists():
    os.mkdir(configs_dir)

config_script_name = os.path.join(configs_dir,"config_"+settings["data"]["name"]+".sh")    
enc = batch_recon.dictionary_prep(settings)
with open(config_script_name, 'w') as f:
    script = template
    script += "\n"
    script += f"srun -N {N} -n 1280 python {os.getcwd()}/backend/batch_recon.py"
    script += " '" + enc + "'"
    f.write(script)
    f.close()
    
# list configsDir
print(f"Contents of batch jobs configs directory: {configs_dir}") 
os.system(f"ls {configs_dir}")
print('')

##### Submit SVMBIR batch job (not currently working)

In [ ]:
# submit batch job config you just created
os.system(f"sbatch {os.path.join(configs_dir,config_script_name)}")
# see all current batch jobs (including this interavtive jupyter session)
print("List of current batch jobs:")
os.system("sqs")

##### To cancel a batch job, run this cell with the right Job ID

In [ ]:
# eq. scancel 2800193
os.system("scancel 0000000")